# Frictionless Product Categorization

- **Author:** Jose Vicente Ruiz Cepeda (jr3660)
- **Course:** COMS W 4995 - Deep Learning for Computer Vision
- **Assignment**: Final project

## Context

Explain some of the context of the problem here.

## Code

### Imports

In [5]:
import keras # Keras 1.2.2 assumed.
from keras import optimizers

from keras.models import Model

from keras.applications import ResNet50
from keras.applications import InceptionV3
from keras.applications import Xception
from keras.applications import VGG16
from keras.applications import VGG19

from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.inception_v3 import preprocess_input

from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Flatten

from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Required to avoid errors with the images.
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

%matplotlib inline

Using TensorFlow backend.


### Constants

In [16]:
CLASS_NAMES = {
    0: 'photo',
    1: 'electronics',
    2: 'events',
    3: 'instruments',
    4: 'tools',
    5: 'sports',
    6: 'caravans',
    7: 'others'
}

### Part I - Categories

In [60]:
train_data_dir = './data/train'
validation_data_dir = './data/validation'
test_data_dir = './data/test'

In [72]:
class Categorizer:
    def __init__(self, num_classes, num_dense_layer_units=512, dropout=0.5):
        self.num_classes = num_classes
        self.num_dense_layer_units = num_dense_layer_units
        self.dropout = dropout
        
    def build_model(self, ImagenetModel=ResNet50, verbose=False):
        # Keras models are functions, not classes, so we have to check which one is it like this.
        if ImagenetModel.func_name == 'InceptionV3' or ImagenetModel.func_name == 'Xception':
            self.img_width, self.img_height = 299, 299
        else:
            self.img_width, self.img_height = 224, 224

        # First, let's load the model with ImageNet weights and without the top layer.
        # This will take some time the first time, since the weights have to be downloaded.
        model = ImagenetModel(
            weights='imagenet',
            include_top=False,
            input_shape=(self.img_width, self.img_height, 3)
        )

        # Now, let's create the top layers adapted to our problem. For that we use the
        # Functional API of Keras. (https://keras.io/getting-started/functional-api-guide/)
        x = model.output
        x = Flatten(input_shape=model.output_shape[1:])(x)
        x = Dense(self.num_dense_layer_units, activation='relu')(x)
        x = Dropout(self.dropout)(x)
        preds = Dense(self.num_classes, activation='softmax')(x)
        self.num_top_layers = 4

        # Combine both models to get the final one.
        self.model = Model(model.input, preds)

        if verbose:
            self.model.summary()

        return self
    
    def compile_model(self, optimizer=optimizers.Adam(lr=0.001)):
        self.model.compile(
            loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy']
        )
            
        return self
    
    def fine_tune(self,
                  class_names,
                  train_data_dir,
                  validation_data_dir,
                  batch_size=16,
                  num_only_top_epochs=10,
                  num_whole_model_epochs=40):
        
        # Augmentation configurations for training and validation.
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)
        validation_datagen = ImageDataGenerator(rescale=1./255)

        # Build the generators.
        train_generator = train_datagen.flow_from_directory(
            train_data_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=batch_size,
            classes=class_names,
            class_mode='categorical')

        validation_generator = validation_datagen.flow_from_directory(
            validation_data_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=batch_size,
            classes=class_names,
            class_mode='categorical')
        
        # Freeze everything except the top layers, before training.
        for layer in self.model.layers[:-self.num_top_layers]:
            layer.trainable = False
        
        self.model.fit_generator(
            train_generator,
            samples_per_epoch=train_generator.n,
            nb_epoch=num_only_top_epochs,
            validation_data=validation_generator,
            nb_val_samples=validation_generator.n
        )
        
        # Unfreeze everything and train for some more epochs.
        for layer in self.model.layers:
            layer.trainable = True
            
        self.model.fit_generator(
            train_generator,
            samples_per_epoch=train_generator.n,
            nb_epoch=num_whole_model_epochs,
            validation_data=validation_generator,
            nb_val_samples=validation_generator.n
        )
        
        return self

In [73]:
categorizer = Categorizer(len(CLASS_NAMES)). \
    build_model(Xception). \
    compile_model()

In [ ]:
categorizer.fine_tune(
    CLASS_NAMES.values(),
    train_data_dir,
    validation_data_dir
)

# Sandbox

In [ ]:
img_path = '/Users/Josevi/product_images/photo/9760-1.jpg'
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [ ]:
imgplot = plt.imshow(mpimg.imread(img_path))
plt.show()

In [ ]:
%time
preds = model.predict(x)
print preds
#print('Predicted:', decode_predictions(preds))

In [ ]:
def show_sample(X, y, prediction=-1):
    im = X
    print y
    #y = np.flip(y, axis=0)
    y_label = class_name[np.nonzero(y)[0][0]]
    plt.imshow(im)
    if prediction >= 0:
        plt.title("Class = %s, Predict = %s" % (y_label, class_name[prediction]))
    else:
        plt.title("Class = %s" % (y_label))

    plt.axis('on')
    plt.show()

In [ ]:
for X_batch, Y_batch in train_generator:
    for i in range(len(Y_batch)):
        show_sample(X_batch[i, :, :, :], Y_batch[i])
    break